### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [42]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from path import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [43]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [44]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [45]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [46]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [47]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

# Cluster Cryptocurrencies with K-means

In this section, you will use the K-Means algorithm with a given value for `k` to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters (`n_clusters=4`). 

2. Fit the K-Means model using the scaled data.

3. Predict the clusters to group the cryptocurrencies using the scaled data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the scaled data with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_14d"` and `y="price_change_percentage_1y"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [48]:
# Initialize the K-Means model with four clusters
model = KMeans(n_clusters=4)

In [49]:
# Fit the K-Means model using the scaled data
#fit the model
model.fit(df_market_data)
#the results output 
KMeans(n_clusters=4)

KMeans(n_clusters=4)

In [50]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters_k4 = model.predict(df_market_data)

# View the resulting array of cluster values.
print(crypto_clusters_k4)

[0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 1 0 2 2 3
 0 0 0 2]


In [51]:
# Note: The code for this step is provided for you.

# Add a new column to the DataFrame with the predicted clusters with k=4
df_market_data_scaled["crypto_cluster_k4"] = crypto_clusters_k4

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,crypto_cluster_k4
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [52]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_14d"` and `y="price_change_percentage_1y"`.
# Group the results by the clusters using `by="crypto_cluster_k4".
# Set the hover to the coin id using `hover_cols=["coin_id"]`.

df_market_data_scaled.hvplot.scatter(
    x="price_change_percentage_14d",
    y="price_change_percentage_1y",
    by="crypto_cluster_k4",
    hover_cols=["coin_id"],
    marker=["hex", "square", "cross", "inverted_triangle", "triangle"],
)

:NdOverlay   [crypto_cluster_k4]
   :Scatter   [price_change_percentage_14d]   (price_change_percentage_1y,coin_id)

---

# Find the Best Value for k

In this section, you will use the elbow method to find the best value for k.

1. Code the elbow method algorithm to find the best value for k. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of k to visually identify the optimal value for k.

3. Answer the following question: What is the best value for k?

In [53]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

In [54]:
# Create an empy list to store the inertia values
inertia = []

In [55]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inirtia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)

In [56]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [57]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

#### 3. Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** 5

---

# Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

6. Initiate a new  K-Means algorithm using the PCA DataFrame to group the cryptocurrencies. Set the `n_components` parameter equal to the best value for `k` found before. View the resulting array. 

7. For further analysis, add the following columns to the DataFrame with the PCA data. Review the resulting DataFrame once the additional columns have been added. Make sure to do the following:

    - From the original DataFrame, add the `price_change_percentage_1y` and `price_change_percentage_14d` columns.

    - Add a column with the predicted cluster values identified using a k value of 4. (The predicted cluster values were calculated in the “Cluster Cryptocurrencies with K-means” section.)

    - Add a column with the predicted cluster values identified using the optimal value for k. 


In [58]:
# Create a PCA model instance and set `n_components=3`.
pca=PCA(n_components=3)

In [59]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
market_pca_data[:5]

array([[-0.46316554, -0.95162232,  0.49628841],
       [-0.43539594, -0.54181334,  0.96834396],
       [-0.54508126,  0.05744219, -0.60656923],
       [-0.60301803,  0.10069339, -0.44473651],
       [-0.69940568, -2.24894113,  1.87306371]])

In [60]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_


array([0.37403031, 0.32182581, 0.16420848])

Answer the following question: What is the total explained variance of the three principal components?

**Question** What is the total explained variance of the three principal components?

**Answer** The explained varience states that all three componenets contain 86 percent of the original information

In [61]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(
    market_pca_data, 
    columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original data
df_market_data_pca["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index("coin_id")

# Display sample data
df_market_data_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.463166,-0.951622,0.496288
ethereum,-0.435396,-0.541813,0.968344
tether,-0.545081,0.057442,-0.606569
ripple,-0.603018,0.100693,-0.444737
bitcoin-cash,-0.699406,-2.248941,1.873064


In [62]:
# Initiate a new  K-Means algorithm using the PCA DataFrame to group 
# the cryptocurrencies. Set the `n_components` parameter equal to 
# the best value for `k` found before. View the resulting array. 

# Initialize the K-Means model
model = KMeans(n_clusters=4)

# Fit the model
model.fit(df_market_data_pca)

# Predict clusters
crypto_clusters_k5 = model.predict(df_market_data_pca)

# View the resulting array
crypto_clusters_k5

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [63]:
# Note: The code for the following step has been provided for you.

# For further analysis, add the following columns to the DataFrame 
# with the PCA data. Review the resulting DataFrame once the additional 
# columns have been added. Make sure to do the following:
# - From the original DataFrame, add the `price_change_percentage_1y` and `price_change_percentage_14d` columns.
# - Add a column with the predicted cluster values identified using a k value of 4. (The predicted cluster values were calculated in the “Cluster Cryptocurrencies with K-means” section.)
# - Add a column with the predicted cluster values identified using the optimal value for k. 


# Add the price_change_percentage_1y column from the original data
df_market_data_pca["price_change_percentage_1y"] = df_market_data["price_change_percentage_1y"]

# Add the price_change_percentage_14d column from the original data
df_market_data_pca["price_change_percentage_14d"] = df_market_data["price_change_percentage_14d"]

# Add a new column to the DataFrame with the predicted clusters using the best value of k
df_market_data_pca["crypto_cluster_k5"] = crypto_clusters_k5

# Add a new column to the DataFrame with the predicted clusters using k=4
df_market_data_pca["crypto_cluster_k4"] = crypto_clusters_k4

# Display sample data
df_market_data_pca.head()

,PC1,PC2,PC3,price_change_percentage_1y,price_change_percentage_14d,crypto_cluster_k5,crypto_cluster_k4
coin_id,,,,,,,
bitcoin,-0.463166,-0.951622,0.496288,37.51761,6.57509,3,0
ethereum,-0.435396,-0.541813,0.968344,101.96023,4.80849,3,0
tether,-0.545081,0.057442,-0.606569,0.01954,0.00640,0,0
ripple,-0.603018,0.100693,-0.444737,-16.60193,2.24984,0,0
bitcoin-cash,-0.699406,-2.248941,1.873064,14.49384,14.75334,3,0


---

# Step 6: Plot Results

In this section, you will visually analyze the cluster analysis results after using the optimization techniques.

1. Use the PCA data to create two scatter plots using hvPlot by setting `x="price_change_percentage_14d"` and `y="price_change_percentage_1y"`. Make sure to do the following: 

    - In the first plot, color the plot points by the cluster values identified using a k value of 4.

    - In the second plot, color the plot points by the cluster values identified using the optimal value for k.

    - In both plots, add the crypto name by sing the `hover_cols` parameter to identify the cryptocurrency represented by each data point. 

2. Be sure to professionally style and format the plots so that the visualizations can be easily read. 

3. Answer the following question: What value of k creates the most accurate clusters of cryptocurrencies, grouped by profitability?


In [72]:
# Create a scatter plot for the Crypto Clusters using k=4 data.
# Use the PCA data to create a scatter plot with hvPlot by setting 
# x="price_change_percentage_14d" and y="price_change_percentage_1y".
# Group by the clusters using `by="crypto_cluster_k4".
# Set the hover colors to the coin id with `hover_cols=["coin_id"]
# Create a descriptive title for the plot using the title parameter.
df_market_data_pca.hvplot.scatter(
    x="price_change_percentage_14d",
    y="price_change_percentage_1y",
    by="crypto_cluster_k4"
)

:NdOverlay   [crypto_cluster_k4]
   :Scatter   [price_change_percentage_14d]   (price_change_percentage_1y)

In [70]:
# Create a scatter plot for the Crypto Clusters using k=5 data.
# Use the PCA data to create a scatter plot with hvPlot by setting 
# x="price_change_percentage_14d" and y="price_change_percentage_1y".
# Group by the clusters using `by="crypto_cluster_k5".
# Set the hover colors to the coin id with `hover_cols=["coin_id"]
# Create a descriptive title for the plot using the title parameter.
df_market_data_pca.hvplot.scatter(
    x="price_change_percentage_14d",
    y="price_change_percentage_1y",
    by="crypto_cluster_k5"
)

:NdOverlay   [crypto_cluster_k5]
   :Scatter   [price_change_percentage_14d]   (price_change_percentage_1y)

In [71]:
# Compare both scatter plots
df_market_data_pca_5 * df_market_data_pca_4

NameError: name 'df_market_data_pca_5' is not defined

Answer the following question: What value of k creates the most accurate clusters of cryptocurrencies, grouped by profitability?

**Question:** What value of `k` seems to create the most accurate clusters to group cryptocurrencies according to their profitability?

**Answer:** YOUR ANSWER HERE!